### Periodic Reprots

1. Reports generated in copertes on repetedly weekly or bi weekly.
2. You have to inform more than 200 project owners for there existing task.
3. Analytical reports or paper based dashboards


In [1]:
# Import of necessay libraries , 
import pandas as pd
import plotly.graph_objects as go
import numpy as np
import seaborn as sns
import os
import datetime as dt
import matplotlib.pyplot as plt
import docx
import sys
from PIL import Image
from docx.shared import Inches
from docx.enum.text import WD_ALIGN_PARAGRAPH
import sqlite3
import datetime
import plotly.express as px

In [2]:
# ==============Change it =========================
conn = sqlite3.connect('C:/Users/Seder/Desktop/KPI_Work/Databasis/initiatives_reporting_Testing.db')
# 

In [3]:
# I need these constant terms to be used in reports
to_days = dt.date.today()
to_days = pd.to_datetime(to_days)
next_15days = to_days + datetime.timedelta(days=15) 

In [4]:
allDataHeader = pd.read_sql_query('Select * From DataHeader', conn, parse_dates=['Due Date', 'Completion Date', 'Timeline - Start', 'Timeline - End', 'Dated'])

In [7]:
allDataHeader.to_clipboard()
lastDate = allDataHeader['Dated'].max()

In [6]:
reccol = ['Name', 'Code', '#', 'Accountable', 'WT%', 'Cost Impact', 'Due Date', 'Overdue', 'Status', 'Completion Date', 
          'Planned%', 'Actual %', 'Duration', 'Timeline - Start',
          'Timeline - End', 'Initiative Type', 'Dated']

allDataHeader = allDataHeader[allDataHeader.columns[allDataHeader.columns.isin(reccol)]]
# allDataHeader = allDataHeader[allDataHeader['Dated'] == lastDate]

In [29]:
allDataHeader.value_counts(['Dated'])

Dated     
2023-10-15    14
2023-10-08     7
Name: count, dtype: int64

In [30]:
allDataItem = pd.read_sql_query('Select * From DataItem', conn, parse_dates=['Timeline - Start','Timeline - End', 'Due Date', 'Completion Date', 'Dated'])
allDataItem = allDataItem[allDataItem['Dated'] == lastDate]

allHrInitiatives = pd.read_sql_query('Select * From HrInitiatives', conn, parse_dates=['Dated','Timeline - Start_Item', 'Timeline - End_Item', 'Due Date_Item', 'Completion Date_Item', 'Due Date_Head', 'Completion Date_Head', 'Timeline - Start_Head', 'Timeline - End_Head' ])

reccolallHrInitiatives = ['Name_Item', 'Owner', 'Team Member_Item', 'Weight',
                           'Timeline - Start_Item', 'Timeline - End_Item', 'Due Date_Item',
                           'Status_Item', 'Completion Date_Item', 'Planned%_Item', 'Actual %_Item',
                           'Project Name', 'Code',  'Due Date_Head', 'Overdue', 'Status_Head', 'Completion Date_Head',
                           'Planned%_Head', 'Actual %_Head', 'Duration', 'Timeline - Start_Head',
                           'Timeline - End_Head', 'Initiative Type', 'Time Buffer', 'Dated']

allHrInitiatives = allHrInitiatives[allHrInitiatives.columns[allHrInitiatives.columns.isin(reccolallHrInitiatives)]]
HR_Initiatives2 = allHrInitiatives[allHrInitiatives['Dated'] != lastDate]
allHrInitiatives = allHrInitiatives[allHrInitiatives['Dated'] == lastDate]

In [31]:
HR_Initiatives1 = HR_Initiatives2.copy()

In [33]:
HR_Initiatives1 = HR_Initiatives2.copy()
Report = HR_Initiatives2['Project Name'].unique()

for i in Report:
    doc = docx.Document()
    section = doc.sections[0]
    header = section.header
    paragraph = header.paragraphs[0]
    paragraph.text = "Seder Logo\tPMO Department Initiatives Report\t PMO Logo"
#     header.add_picture('C:/Users/Administrator/Desktop/Logs/PMO/LOGO White.png')
    paragraph.style = doc.styles["Header"]
 
    temp = HR_Initiatives1.copy()
    temp = HR_Initiatives1[HR_Initiatives1['Project Name'] == i]

#   Sub header Section of the reports ##
    doc.add_heading('\t\t'+i, level=1)
    doc.add_heading("Report Date "+str(to_days.date())+"\t\t\t\t\t\tInitiative Type :"+ str(temp['Initiative Type'].unique()[0]), level=9)

#   Initiative owners and Team members
    Owners = str(temp['Owner'].dropna().unique())
    Owners = Owners.replace('[','').replace(']','').replace('\'','')
    doc.add_heading('Initiative Owner: '+ Owners, level=9)
    

    Team_members = str(temp['Team'].dropna().unique())
    Team_members = Team_members.replace('[','').replace(']','').replace('\'','')
    doc.add_heading('Initiative Members: '+ Team_members, level=9)
    
    
#  doc.add_heading("Initiative Type :"+ str(temp['Initiative Type'].unique()[0]), level=1)
    
    # *********Code block for Overall Porgress ******************************

    pics = []
       
    # Code for Visiual number 1 Days
    doc.add_heading('\t\t\tCurrent Progress ', level=2)
    doc.add_heading('', level=2)
    
    
    O = temp[['Timeline - Start_Head','Due Date_Head']]
    O = O.copy()
    O['Due Date_Head'] = pd.to_datetime(O['Due Date_Head'])
    O['Planned Days'] =  O['Due Date_Head'] - O['Timeline - Start_Head']
    O['Days to Date'] =  to_days - O['Timeline - Start_Head']
    O['Left Days'] = O['Planned Days'] - O['Days to Date']

    planned_days = O.iat[0,3].days
    DaystoDate = O.iat[0,4].days

    fig = go.Figure(go.Indicator(
        mode = "number+delta",
        value = planned_days,
        number = {'prefix': "Days "},
        delta = {'reference': DaystoDate},
        domain = {'x': [0, 1], 'y': [0, 1]}))

    fig.update_layout(paper_bgcolor = "lightgray",
       title="Progress by Days",
        title_font_color="green",
        title_font_size=35,
        title_x=0.5,
       
    )

    fig.write_image('Images/Plotly/'+i+'days.png')
#     doc.add_picture('Images/Plotly/'+i+'days.png', width=Inches(4.25))
    pics.append('Images/Plotly/'+i+'days.png')
    
 
    temp['variance'] = temp['Planned%_Head'] - temp['Actual %_Head']
    o = temp[['Planned%_Head','Actual %_Head']]
    Actual = o.iat[1,1]
    Planned = o.iat[0,0]
    Actual, 
    Planned

    fig = go.Figure(go.Indicator(
        mode = "number+gauge+delta",
        gauge = {'shape': "bullet"},
        delta = {'reference': Planned},
        value = Actual,
        domain = {'x': [0.1, 1], 'y': [0.2, 0.9]},
        title = {'text': "Actual"}))

    fig.update_layout(title="Planned Target till Date Is "+ str(Planned),
        title_font_family="Times New Roman",
        title_font_color="green",
        title_font_size=35,
        legend_title_font_color="green",
        title_x=0.5
    )
   
        
#     fig.update_layout(paper_bgcolor = "lightgray")
    fig.write_image('Images/Plotly/'+i+'vari.png')
#     doc.add_picture('Images/Plotly/'+i+'vari.png', width=Inches(4.25))
    pics.append('Images/Plotly/'+i+'vari.png')

#  Line below is subjected to Delete    
#     doc.add_heading("The current progress varience is "+str(projectVar[0]), level=5 )


   
    
    
    
# Code for visiual Number lookahed and overall

    #OverDue
    OverDue = temp[(temp['Due Date_Item'] <= to_days)&(temp['Status_Item'] != 'Done')]
    OverDue = OverDue[['Name_Item', 'Timeline - End_Item', 'Status_Item']]
    OverDue['Status_Item'] = OverDue['Status_Item'].fillna('Not Started')
    OverDue['Timeline - End_Item'] = OverDue['Timeline - End_Item'].dt.date
    OverDue.shape[0]

    fig = go.Figure(go.Indicator(
        mode = "number",
        value = OverDue.shape[0],
    #     number = {'prefix': "Overdue Tasks "},
    #     delta = {'reference': DaystoDate},
        domain = {'x': [0, 1], 'y': [0, 1]}))

    fig.update_layout(paper_bgcolor = "lightgray",
                      title="Overdue Tasks",
                      title_font_size=35,
                      title_font_color="green",
                      title_x=0.5

                     )
    fig.write_image('Images/Plotly/'+i+'overdue.png')
#     doc.add_picture('Images/Plotly/'+i+'overdue.png', width=Inches(4.25))
    pics.append('Images/Plotly/'+i+'overdue.png')
    # fig.show()


    #  LookAhead Items
    check = ['Working on it', np.nan]
    lookAhead = temp[(temp['Due Date_Item'] >= to_days)&(temp['Due Date_Item'] <= next_15days)&(temp['Status_Item'].isin(check) )]
    lookAhead = lookAhead[['Name_Item', 'Timeline - End_Item', 'Status_Item']]
    lookAhead['Timeline - End_Item'] = lookAhead['Timeline - End_Item'].dt.date
    lookAhead.shape[0]

    fig = go.Figure(go.Indicator(
        mode = "number",
        value =lookAhead.shape[0],
        domain = {'x': [0, 1], 'y': [0, 1]}))

    fig.update_layout(title="Lookahed Tasks",
                      title_font_size=35,
                      title_font_color="green",
                      title_x=0.5

                     )
    fig.write_image('Images/Plotly/'+i+'lookahed.png')
#     doc.add_picture('Images/Plotly/'+i+'lookahed.png', width=Inches(4.25))
    pics.append('Images/Plotly/'+i+'lookahed.png')
    
    images = [Image.open(x) for x in pics]
    widths, heights = zip(*(i.size for i in images))

    total_width = sum(widths)
    max_height = max(heights)

    new_im = Image.new('RGB', (total_width, max_height))

    x_offset = 0
    for im in images:
      new_im.paste(im, (x_offset,0))
      x_offset += im.size[0]

    new_im.save('Images/Plotly/imagecombo.jpg')
    doc.add_picture('Images/Plotly/imagecombo.jpg', width=Inches(6.25), height=Inches(1.25))
    

    
    
    
# Code for Privious overall project
    doc.add_heading('\t\t Overall Project status from Start to End', level=2)
    
    temp[['Timeline - Start_Item', 'Timeline - End_Item' , 'Completion Date_Item']].plot(style='*-', figsize=(10, 4), fontsize=8, title=i, grid=True, xlabel='Task arrange in Numbers accordingly', ylabel='Date From Start till End')
    plt.savefig('Images/Matplot/'+i+'.png')
    
    doc.add_picture('Images/Matplot/'+i+'.png', width=Inches(6.25))
    doc.add_heading('Project Detail View from start untill today can be observed. Star Marked are Tasks According to there Planned, Start and End Date and Actual Completion. Variance between the Planned Closed date and Actual Closed Date are clearly visible. ', level=5)

    
    #     Delayed Items

    o1 = temp[['Name_Item', 'Timeline - End_Item','Completion Date_Item']]
    o1 = o1.copy()
    o1['Extra Days'] =  o1['Completion Date_Item'] - o1['Timeline - End_Item']
    o1 = o1[(o1['Extra Days'].notna())]
    o1 = o1[(o1['Extra Days'] >= pd.to_timedelta(1,unit='D'))]
    
    if o1.shape[0] != 0:
        doc.add_heading('Progress Delayed Items ', level=2)
        doc.add_heading('Following items effected the progress performence ', level=5)    
        t3 = doc.add_table(o1.shape[0]+1, o1.shape[1], style = 'Table Grid')

        # add the header rows.
        for j3 in range(o1.shape[-1]):
            t3.cell(0,j3).text = o1.columns[j3]

        # add the rest of the data frame
        for m3 in range(o1.shape[0]):
            for j3 in range(o1.shape[-1]):
                t3.cell(m3+1,j3).text = str(o1.values[m3,j3])

     
    #OverDue

    doc.add_heading("Comments ", level=2)    
    if OverDue.shape[0] <= 2:
        doc.add_heading('Over Due Items Needs Attention by the team', level=5)
    elif OverDue.shape[0] > 3:
        doc.add_heading('Over Due Items are higher and Urgent Attention is needed by the team', level=5)        
    
#   doc.add_heading(filename, 0)
    if OverDue.shape[0] > 0:

        doc.add_heading('Over Due Items', level=3)

        t = doc.add_table(OverDue.shape[0]+1, OverDue.shape[1], style = 'Table Grid')

        # add the header rows.
        for j in range(OverDue.shape[-1]):
            t.cell(0,j).text = OverDue.columns[j]

        # add the rest of the data frame
        for m in range(OverDue.shape[0]):
            for j in range(OverDue.shape[-1]):
                t.cell(m+1,j).text = str(OverDue.values[m,j])

#  LookAhead Items
    if lookAhead.shape[0] != 0:
        doc.add_heading('Look Ahead Items', level=3)

    
        t1 = doc.add_table(lookAhead.shape[0]+1, lookAhead.shape[1], style = 'Table Grid')

        # add the header rows.
        for k in range(lookAhead.shape[-1]):
            t1.cell(0,k).text = lookAhead.columns[k]    

        # add the rest of the data frame
        for n in range(lookAhead.shape[0]):
            for k in range(lookAhead.shape[-1]):
                t1.cell(n+1,k).text = str(lookAhead.values[n,k])


    doc.save(i+str(to_days.date())+'.docx')


KeyError: 'Team'